# Model Deployment

At this step we will demonstrate how to transition from experimenting in SageMaker Studio notebook to pushing the model to production. We will show how to approve the model version in the staging environment and then transition it to a production environment. The example below focuses on the real-time inference case.

These are the main steps that will be undertaken by the data scientist to deploy a model using the MLOPs platform:

1. Update the configuration file in SageMaker Studio to define how the model will be built.
2. Commit your changes in the project directory in SageMaker Studio to a local development branch. 
3. Push the local development branch to the Code Commit repository associated with the project.
3. Create a merge request to merge the development branch with the main branch. This will initiate a Code Pipeline to build the model using SageMaker Pipelines.
4. Approve model version in SageMaker Studio. This will initiate a Code Pipeline to deploy the model in the staging environment. Note, that in the multi-account deployment the staging environment will be a separate AWS account. In the single account deployment the staged model will be deployed in the same account. 
5. Approve deployment to production in Code Pipeline to create a production endpoint. Note, that in the multi-account deployment the production environment will be a separate AWS account. In the single account deployment the model in production will be deployed in the same account. 


Make sure to log into AWS Console with the Data Science User role to perform the steps below. Follow [AWS console](https://signin.aws.amazon.com/switchrole) link to assume the Data Science Admin role. The role has been created for you during the MLOps platform deployment. The role name has the following pattern:  **region-datascientist-account_id**. You might find it helpful to first navigate to IAM in AWS Console in order to retrieve the role name and then assume it using [AWS console](https://signin.aws.amazon.com/switchrole) link.

## Trigger Model Building

When a data scientist completed model experimentation and is ready to push it to production, they need to modify a parameter file associated with the project to provide the key information about the model and the data necessary for building the model. 

1. During the **Model Development** stage you were asked to take a note of the project S3 bucket. This information can also be retrieved by running the command `cat .project_params.yaml` in SageMaker Studio terminal.

2. Once the S3 bucket name is retrieved, modify the `model_config.yaml` by replacing the placeholder in the file for the S3 bucket under data_config/inputs/data. At this point the data scientist has been working with files in the cloned project repository. Ensure that you modify the `model_config.yaml` in your project repository. 

![ds_model_build2.png](images/ds_model_build.gif)

This file `model_config.yaml` defines how the model training pipeline is configured and the parameters in the file map to SageMaker Python SDK. The mapping is outlined in the table below:

<center>

model_config.yaml parameter | SageMaker Python SDK
------------------------------|---------------------
`preprocess_config` | [Sagemaker Preprocessing job](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor)
`train_config` | [Sagemaker training job](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator)
`inference_config` | [Sagemaker Endpoint](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html?highlight=RegisterModel#sagemaker.workflow.step_collections.RegisterModel)
`framework`, `framework_version` | used for finding the base docker images<br />for ML framework, such as sklearn, pytorch, tensorflow, etc. <br />Refer to [Image URIs](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html)
`data_config` | [data input](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ProcessingInput)
`bias_config` | [bias config](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html?highlight=BiasConfig#sagemaker.clarify.BiasConfig)

</center>

3. Once the changes to the `model_config.yaml` are made, you can commit your changes to the `dev` branch, and push the branch to the Code Commit repository associated with the project.

 ![ds_model_build4.png](images/ds_model_build4.gif)

4. Navigate to the [Code Commit console](https://console.aws.amazon.com/codecommit/home) page and click on `Create pull Request` from the `dev` (source) to the `main` (destination) branch. 

![ds_model_build5.png](images/ds_model_build5.png)

![ds_model_build6.png](images/ds_model_build6.png)

Provide merge title and an optional description. 

![ds_model_build7.png](images/ds_model_build7.png)

Once the branches are merged successfully it will automatically trigger the model building via using Code Pipeline and SageMaker Pipelines.

![ds_model_build8.png](images/ds_model_build8.png)

![ds_model_build9.png](images/ds_model_build9.png)

5. The status of the Code Pipeline can be monitored by navigating to [Code Pipeline Console](https://console.aws.amazon.com/codepipeline/home). The name of the pipeline is **projectname_build_pipeline**. 

![ds_model_build10.png](images/ds_model_build10.png)

![ds_model_build11.png](images/ds_model_build11.png)

6. The model is built using SageMaker Pipelines. The progress of this step can be monitored in SageMaker Studio as shown below: 

![ds_model_build12.gif](images/ds_model_build12.gif)

## Approve Model Deployment

1. Once the model building pipeline is completed, the new model will be available through SageMaker Studio. You can navigate to SageMaker resources and select `Model Registry`. In the `Model Registry` you will be able to click on the `Model group name` that corresponds to your project. This will show all model versions available for this project together. 

2. Initial model status will be `Pending`. At this stage you can update the status to `Approved`. This action will trigger execution of another Code Pipeline **projectname_deploy_pipeline**. The purpose of this execution is to deploy the model to a staging environment. Please note, that in multi-account MLOPs deployment the staging environment will be a separate AWS account. 

![ds_model_deployment.gif](images/ds_model_deployment.gif)

3. To monitor the status of **projectname_deploy_pipeline** Code Pipeline navigate to the [Code Pipeline Console](https://console.aws.amazon.com/codepipeline/home).

![ds_model_deployment5.png](images/ds_model_deployment5.png)

![ds_model_deployment6.png](images/ds_model_deployment6.png)

![ds_model_deployment7.png](images/ds_model_deployment7.png)

4. Once the pipeline execution is completed the staging endpoint **projectname-staging-endpoint** will be `InService`. The status of the endpoint can be verified by navigating to SageMaker console, `Inference` and then `Endpoints`. 

![ds_model_deployment8.png](images/ds_model_deployment8.png)

5. Next, the model can be promoted to the production environment. In the multi-account deployment it will be a different AWS account from the staging account. This is accomplished through a manual approval step in the execution of **projectname_deploy_pipeline** Code Pipeline which at this stage is paused in the `ApproveDeployment` state. The data scientist will click on the `Review` button in the step `ApproveDeployment` and optionally provide comments. This action will un-pause the Code Pipeline execution and result in the deployment of the production endpoint with the model monitoring enabled. 

![ds_model_deployment9.png](images/ds_model_deployment9.png)

![ds_model_deployment10.png](images/ds_model_deployment10.png)

6. The status of the production endpoint can be monitored by navigating to SageMaker console, `Inference` and then `Endpoints`. The name of the production endpoint is **project-prod-endpoint**.

![ds_model_deployment11.png](images/ds_model_deployment11.png)
